<a href="https://colab.research.google.com/github/jaxtew/med-lm/blob/main/med_lm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
!pip install langchain > /dev/null
!pip install OpenAI > /dev/null
!pip install tiktoken > /dev/null

In [8]:
from langchain.llms import OpenAI

In [60]:
import os
from getpass import getpass
key = getpass()
os.environ['OPENAI_API_KEY'] = key

··········


In [62]:
MODEL_NAME = "text-davinci-003"

In [28]:
from langchain import PromptTemplate, LLMChain

In [63]:
with open("progress_note.txt", 'r') as notef:
  note = notef.read()

816

In [ ]:
OpenAI(model_name=MODEL_NAME).get_num_tokens(note)

In [55]:
questions = {
  '1': 'How many self-limited or minor problems assessed?',
  '2': 'How many stable chronic problems assessed?',
  '3': 'How many acute but uncomplicated/minor problems assessed?',
  '4': 'How many chronic illnesses with exacerbation/progression/side effects of treatment assessed?',
  '5': 'How many undiagnosed new problems with uncertain prognosis assessed?',
  '6': 'How many acute illnesses with systemic symptoms assessed?',
  '7': 'How many acute injuries with complications assessed?',
  '8': 'How many chonic illnesses with SEVERE exacerbation/progression/side effects of treatment assessed?',
  '9': 'How many problems pose a threat to life or permanent loss of bodily function?'
}

formatted_questions = '\n'.join([f"{k}. {v}" for k, v in questions.items()])

In [67]:
from pydantic import BaseModel, Field

class Questions(BaseModel):
  one: str = Field(description="number of self-limited or minor problems assessed")
  two: str = Field(description="number of stable chronic problems assessed")
  three: str = Field(description="number of acute but uncomplicated/minor problems assessed")
  four: str = Field(description="number of chronic illnesses with exacerbation/progression/side effects of treatment assessed")
  five: str = Field(description="number of undiagnosed new problems with uncertain prognosis assessed")
  six: str = Field(description="number of acute illnesses with systemic symptoms assessed")
  seven: str = Field(description="number of acute injuries with complications assessed")
  eight: str = Field(description="number of chronic illnesses with severe exacerbation/progression/side effects of treatment assessed")
  nine: str = Field(description="number of problems that pose a threat to life or permanent loss of bodily function")



In [ ]:
class AnalysisPipeline:
  def get_pydantic_parser(self):
    raise NotImplementedError("Abstract")
  
  def compute_category(self, BaseModel)

In [68]:
from langchain.output_parsers import PydanticOutputParser

parser = PydanticOutputParser(pydantic_object=Questions)

In [69]:
template = """Patient progress note: {note}

Questions: 
{questions}
"""

prompt = PromptTemplate(template=template, 
                        input_variables=['note'], 
                        partial_variables={"questions": parser.get_format_instructions()})

In [70]:
chain = LLMChain(prompt=prompt, llm=OpenAI(model_name=MODEL_NAME))

In [73]:
output = chain(inputs={"note": note})

out_map = parser.parse(output['text'])